In [1]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm

from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.simplefilter("ignore")

In [2]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    save_dir = "G:/マイドライブ/signate_StudentCup2023/exp/"
    data_dir = "G:/マイドライブ/signate_StudentCup2023/data/"
    filename = "exp066"

In [3]:
# ===================================================================
#  Utils
# ===================================================================
def seed_everything(seed):
    """fix random factors"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)
    

def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [4]:
# ===================================================================
#  Data Loading
# ===================================================================
df = pl.read_csv(CFG.save_dir+f"oof_df_exp065.csv").sort("id")
train = pl.read_csv(CFG.data_dir+"train.csv", columns=["id", "price"])
df = df.join(train, on="id", how="left")
df = df.join(
    pl.read_csv(CFG.save_dir+"kun_exp00052_oof_pred.csv"),
    on="id", how="left"
)

test = pl.read_csv(CFG.save_dir+f"exp065.csv").sort("id")
test = test.join(
    pl.read_csv(CFG.save_dir+"kun_exp00052.csv"),
    on="id", how="left"
)

df.head()

id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18,pred_19,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29,pred_30,pred_31,pred_32,pred_33,pred_34,pred_35,pred_36,pred_37,pred_38,pred_39,price,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,kun_pred_9
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,6881.892301,6517.414858,7476.567746,7052.61059,6809.002174,6465.089852,7714.185588,7010.85559,7184.306903,6364.997182,7501.188408,7678.577536,8374.386963,8105.340176,8644.322072,7985.056344,8628.261509,6528.148787,7798.699012,8063.395359,8981.008775,8613.396141,10717.468387,9447.344769,7984.506685,8272.276158,5876.6578,6016.32209,7374.013382,8496.804984,7400.051675,7502.731399,9046.083334,9790.2077,7999.621081,9179.605316,7404.460231,10863.206534,7674.566326,10945.332692,27587,7118.9443,7627.1987,7084.0015,7043.095,7122.753,7335.231,7350.006,9836.875,8781.688,8747.975
1,3740.634027,3496.012582,3789.907986,3322.844722,3603.935425,3485.623398,3507.497452,3714.057283,3488.353022,3750.770625,3412.19359,3927.363047,3886.662697,3551.474485,3713.3473,3463.093834,3695.630189,3663.237361,3463.024267,3905.364004,3563.378845,4112.248835,3360.300264,3807.072052,3406.217022,3615.842705,4245.317355,4453.437876,4408.398403,3900.820201,3869.869927,3681.464095,3648.063325,4057.324098,3423.298315,4256.8008,3937.424257,4012.205191,3871.318876,4087.226759,4724,3660.8042,3526.6816,3732.6855,3593.2502,3596.9211,3562.2925,3922.8992,3761.7544,3852.5068,3476.0593
2,2954.247573,2735.110086,3288.647709,2893.84021,3253.350282,2885.690295,3202.10377,2799.365349,2989.931642,2920.912332,3059.180374,3198.706345,2764.92988,3237.239548,2756.230854,3233.761663,2927.997318,3258.873698,3235.904715,3139.762054,3016.016944,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537,2846.925805,3310.14365,3429.863903,2884.781039,3109.509196,3182.319352,2966.910201,3300.768016,2975.34178,2996.449911,10931,2883.739,3149.7769,2995.776,3038.0789,2876.831,2982.5576,2787.5325,2911.18,3172.8408,3092.2769
3,8430.949224,8337.416095,9033.353098,8081.40405,8187.453186,8477.54243,8546.771462,8388.923127,8119.18331,8413.930266,8298.903056,8142.383515,8055.886032,8356.561411,8421.656285,8167.994718,8646.470884,8423.019535,8016.7337,8041.633697,8331.325796,7947.686753,8057.31244,8250.578545,8052.200549,8115.355025,8157.420044,7434.588221,8284.727211,7855.624335,8375.564192,8287.367768,8814.043198,8434.204563,8292.250116,8010.910553,8092.32479,8698.940654,8281.208121,7630.436524,16553,8650.627,8943.903,9243.175,8973.261,9158.718,8505.161,8713.231,8287.674,8649.64,8874.921
4,3972.418866,4254.790314,4089.549517,4272.046434,4446.949951,4022.828501,4374.413436,4369.679027,4150.7095,3941.880817,4227.904345,4420.621374,4676.941221,4534.857719,4604.858395,4588.033874,4023.362549,4455.22004,4574.171104,4520.151023,4408.383964,4200.673767,4535.020561,4420.531149,4421.192901,4254.960815,4031.139467,3972.143434,4027.789862,4720.410259,4455.75972,3900.863059,4312.082422,4518.990036,4405.510685,4745.297877,4280.133207,4148.129445,4364.547169,4530.330211,5158,4031.1462,4406.548,4107.919,4059.3975,4229.2666,4429.893,4209.0796,4026.7249,4228.1797,4164.242


In [5]:
# ===================================================================
#  Hill Climbing
# ===================================================================
# https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/369609


def evaluate(current_best_preds, k: int, best_score: float):
    """
    評価関数
    
    preds = current_best_preds*(1-w) + df[models[k]]*w
    において最もスコアが良くなるようにwを探す.
    もしその最も良いスコアがbest_scoreよりも良ければそのスコアと重みを返し、その最も良いスコアがbest_scoreよりも良くなければ元々のbest_scoreと重み0を返す
    """
    best_weight = 0
    for w in np.arange(-0.5, 0.5, 0.001):
        preds = current_best_preds*(1-w) + df[models[k]]*w
        score = get_score(y_true=df["price"], y_pred=preds)
        if score < best_score:
            best_score = score
            best_weight = w
    return best_score, best_weight



stores = dict() # Hill Climbingで得た重りを保存する
orders = []     # Hill Climbingで選ばれたモデルの順番を保存する
scores = dict() # Hill Climbingするときのスコアを保存する

# single modelで最もCVが良いモデルを選択する
for col in df.columns:
    if col not in ["id", "price"]:
        scores[col] = get_score(y_true=df["price"], y_pred=df[col])        
selected_model = min(scores, key=scores.get)
current_best_preds = df[selected_model]
orders.append(selected_model) # 順番を保存
stores[selected_model] = 1    # 重みを保存
models = [col for col in df.columns if col not in ["id", "price"]]

# Start Hill Climbing
i = 0
print(f"[{i}] baseline {selected_model} {min(scores.values())}")

while True:
    
    # 前段階のベストスコア
    best_score = min(scores.values())
    
    # 前段階で選ばれたモデル
    selected_model = min(scores, key=scores.get)
    
    # 重複しないように削除
    models.remove(selected_model)
    
    
    # 残ったモデルの数が0になれば終了
    if len(models) == 0:
        break
    
    # 
    scores, weights = dict(), dict()
    for k in range(len(models)):
        score, weight = evaluate(current_best_preds, k, best_score)
        scores[models[k]] = score
        weights[models[k]] = weight
        
    i += 1
    selected_model = min(scores, key=scores.get)
    print(f"[{i}] add {selected_model}: {min(scores.values())} {weights[selected_model]}")
    best_weight = weights[selected_model]
    stores[selected_model] = best_weight
    orders.append(selected_model)
    current_best_preds = current_best_preds*(1-best_weight) + df[selected_model]*best_weight

[0] baseline pred_12 43.805041424202756
[1] add pred_29: 43.68618295984381 0.47700000000000087
[2] add pred_10: 43.6494565341299 0.2860000000000007
[3] add pred_24: 43.6341430286122 0.21600000000000064
[4] add pred_20: 43.62363269458271 0.13000000000000056
[5] add pred_9: 43.616045519428944 -0.14399999999999968
[6] add pred_28: 43.60765780674308 0.15100000000000058
[7] add pred_1: 43.60388338358262 -0.08199999999999963
[8] add pred_14: 43.598003437354926 0.11800000000000055
[9] add kun_pred_8: 43.59425206119694 -0.09499999999999964
[10] add pred_11: 43.591797195246585 0.07500000000000051
[11] add pred_3: 43.58829582745161 -0.09899999999999964
[12] add pred_5: 43.58566739316292 0.07400000000000051
[13] add pred_13: 43.58378731138246 -0.07099999999999962
[14] add pred_36: 43.58254458766511 0.04900000000000049
[15] add pred_25: 43.58073405013395 -0.06999999999999962
[16] add pred_33: 43.57956912525516 0.057000000000000495
[17] add kun_pred_4: 43.57817333653196 -0.054999999999999605
[18] a

In [6]:
# ===================================================================
#  Check
# ===================================================================
def get_preds(df: pl.DataFrame):
    best_preds = 0
    for exp in orders:
        w = stores[exp]
        best_preds  = best_preds * (1-w) + df[exp] * w
    return best_preds
    
get_score(y_true=df["price"], y_pred=get_preds(df))

43.56453307604806

In [7]:
# ===================================================================
#  oof_df
# ===================================================================
df.with_columns(
    pl.Series(get_preds(df)).alias("pred")
)[["id", "pred"]].write_csv(CFG.save_dir+f"oof_df_{CFG.filename}.csv", has_header=True)

display(df.with_columns(
    pl.Series(get_preds(df)).alias("pred")
)[["id", "pred"]].head())

id,pred
i64,f64
0,8854.693376
1,3869.563539
2,3020.466736
3,8014.619227
4,4522.686418


In [8]:
# ===================================================================
#  test
# ===================================================================
test = test.with_columns(
    pl.Series(get_preds(test)).alias("pred")
)
test.select(["id", "pred"]).write_csv(CFG.save_dir+f"{CFG.filename}.csv", has_header=False)
test.select(["id", "pred"]).head()

id,pred
i64,f64
27532,9673.031004
27533,5585.545628
27534,5595.254657
27535,18194.779659
27536,4416.241293
